In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

spark = SparkSession.builder.appName("Chapter8").getOrCreate()


person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

joinExpr = person['graduate_program'] == graduateProgram['id']

person.withColumnRenamed("id", "empid").join(graduateProgram, joinExpr).select("empid", "name", "spark_status", "degree", "department").show(10, False)

joinType = "inner"

person.withColumnRenamed("id", "empid").join(graduateProgram, joinExpr, joinType).select("empid", "name", "spark_status", "degree").show(5, False)

joinType="outer"

person.join(graduateProgram, joinExpr, joinType).select("*").show(5, False)

joinType = "left_outer"

person.join(graduateProgram, joinExpr, joinType).select("*").show(5, False)

joinType = "right_outer"

person.join(graduateProgram, joinExpr, joinType).select("*").show(5, False)

joinType = "left_semi"

person.join(graduateProgram, joinExpr, joinType).select("*").show(5, False)

joinType = "left_anti"

person.join(graduateProgram, joinExpr, joinType).select("*").show(5, False)

person.withColumnRenamed("id", "personId")\
  .join(sparkStatus, expr("array_contains(spark_status, id)")).show()


+-----+----------------+---------------+-------+---------------------+
|empid|name            |spark_status   |degree |department           |
+-----+----------------+---------------+-------+---------------------+
|0    |Bill Chambers   |[100]          |Masters|School of Information|
|1    |Matei Zaharia   |[500, 250, 100]|Ph.D.  |EECS                 |
|2    |Michael Armbrust|[250, 100]     |Ph.D.  |EECS                 |
+-----+----------------+---------------+-------+---------------------+

+-----+----------------+---------------+-------+
|empid|name            |spark_status   |degree |
+-----+----------------+---------------+-------+
|0    |Bill Chambers   |[100]          |Masters|
|1    |Matei Zaharia   |[500, 250, 100]|Ph.D.  |
|2    |Michael Armbrust|[250, 100]     |Ph.D.  |
+-----+----------------+---------------+-------+

+----+----------------+----------------+---------------+---+-------+---------------------+-----------+
|id  |name            |graduate_program|spark_status   